In [ ]:
# Perform imports
import polars as pl
from datetime import datetime
import altair as alt
from securities_load.securities.polar_table_functions import (retrieve_close_using_currency_tickers_dates, 
                                                            retrieve_watchlists_using_watchlist_type, 
                                                            retrieve_tickers_using_watchlist_code)

In [2]:
pl_df = retrieve_close_using_currency_tickers_dates(
    "USD", ["AAPL", "MSFT", "GOOGL"], "2023-01-01", "2023-10-01")
moving_average_pl_df = pl_df.with_columns(
    pl.col("close").rolling_mean(window_size=10).alias("close_MA10"),
    pl.col("close").rolling_mean(window_size=50).alias("close_MA50"),
)

In [3]:
pl_df.plot.line(
    x="date:T",
    y="close:Q",
    color="ticker:N",
).show()

alt.Chart(...)

In [4]:
line_chart = (
    alt.Chart(pl_df)
    .mark_line(strokeWidth=1)
    .encode(x=alt.X("date:T",
                    title='',
                    axis=alt.Axis(labelAngle=-45, tickColor="#09080B", domainColor="#09080B", grid=False, labelFontSize=10, titleFontSize=12)),
            y=alt.Y("close:Q",
                    title='Close',
                    scale=alt.Scale(zero=False),
                    axis=alt.Axis(format='$.2f', tickColor="#09080B", domainColor="#09080B", gridColor="#9DCDF8", grid=True, labelFontSize=10, titleFontSize=12)
                    ),
            color="ticker:N",
            tooltip=['date:T', 'close:Q'])
    ).properties(title={'text': 'Stock Price Analysis',
                        'subtitle': ['Comparison of multiple tickers'],
                        'fontSize': 16
                },
                width=500,
                height=300,
    )
line_chart

alt.Chart(...)

In [5]:
filter_on = "AAPL"
filtered_df = moving_average_pl_df.filter(pl.col("ticker") == filter_on)
long_data = filtered_df.unpivot(index='date', on=['close', 'close_MA10', 'close_MA50'], variable_name='type', value_name='value')

line_chart = (
    alt.Chart(long_data)
    .mark_line(strokeWidth=1)
    .encode(x=alt.X("date:T",
                    title='',
                    axis=alt.Axis(labelAngle=-45, tickColor="#09080B", domainColor="#09080B", grid=False, labelFontSize=10, titleFontSize=12)),
            y=alt.Y("value:Q",
                    title='Price',
                    scale=alt.Scale(zero=False),
                    axis=alt.Axis(format='$.2f', tickColor="#09080B", domainColor="#09080B", gridColor="#9DCDF8", grid=True, labelFontSize=10, titleFontSize=12)
                    ),
            color="type:N",
            tooltip=['date:T', 'value:Q'])
    ).properties(title={'text': f'Stock Price Analysis for {filter_on}',
                        'subtitle': ['Comparison of multiple tickers'],
                        'fontSize': 16
                },
                width=500,
                height=300,
    )
line_chart

alt.Chart(...)